In [54]:
import requests
import os
import json
import pandas as pd

In [55]:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAIxjZAEAAAAAIktbOv%2Fsv8GGGPhOoY1ZfcmCCik%3DaK1do3V6O' \
               'yehB2URtPahKgjbtgKUhq4B8T72kMt3yxnrVEYUeU'

In [56]:
def create_url(user_id, start_date, end_date, next_token, max_results):
    search_url = "https://api.twitter.com/2/users/{}/tweets".format(user_id)

    query_params = {
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields': 'author_id,id,created_at,lang,text,public_metrics,referenced_tweets,'
                                    'in_reply_to_user_id,conversation_id',
                    'user.fields': 'id,name,username,description,verified',
                    # 'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    # 'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'pagination_token': next_token}
    return search_url, query_params

In [57]:
def create_header(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    # r.headers["User-Agent"] = "v2UserTweetsPython"
    return headers

In [58]:
def connect_to_endpoint(url, headers, params, next_token=None):
    if not next_token:
        params['pagination_token'] = next_token
    # print(params)
    # print(headers)
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [59]:
def main():
    # user_id_lst = []
    # user_id_lst.append('109375565')
    start_lst = ['2022-03-01T00:00:00.000Z',
                  '2022-02-01T00:00:00.000Z',
                  '2022-01-01T00:00:00.000Z']

    end_lst = ['2022-04-01T00:00:00.000Z',
                '2022-03-01T00:00:00.000Z',
                '2022-02-01T00:00:00.000Z']
    user_id = '109375565'
    # start_date = '2022-01-01T00:00:00.000Z'
    # end_date = '2022-03-25T00:00:00.000Z'

    max_results = 10
    total_tweets = 0
    max_request = 2    # max 450 requests per 15mins
    request = 0
    flag = True
    sleep_time = 60*15
    df = pd.DataFrame()
    next_token = None
    headers = create_header(bearer_token)

    while flag:
        for i in range(len(start_lst)):
            next_token = None
            count = 0
            for j in range(max_request):
                url, params = create_url(user_id, start_lst[i], end_lst[i], next_token, max_results)
                json_response = connect_to_endpoint(url, headers, params, next_token)
                request += 1
                print(json.dumps(json_response, indent=4, sort_keys=True))
                if json_response['meta']['result_count'] == 0:
                    print('No data found.')
                    break
                if df.empty:
                  df = pd.DataFrame(json_response['data'])
                else:
                  df = df.append(pd.DataFrame(json_response['data']), ignore_index=True)
                print('**************************************************\n',df.shape, '\n',df)
                result_count = json_response['meta']['result_count']
                next_token = json_response['meta']['next_token']
                count += result_count
                total_tweets += result_count
                print('------------------------------------------------', count, total_tweets)

                if request >= 450:
                    print('sleeping...')
                    time.sleep(sleep_time)
                if not next_token:
                    flag = False
                    print('next token is None')
                    break
        flag = False
    print('df: ', df.shape, df)
    df.to_csv('request_by_user_data.csv')

In [60]:
if __name__ == "__main__":
    main()

200
{
    "data": [
        {
            "author_id": "109375565",
            "conversation_id": "1507378609899524096",
            "created_at": "2022-03-25T18:12:27.000Z",
            "id": "1507420160486903809",
            "in_reply_to_user_id": "264010057",
            "lang": "en",
            "public_metrics": {
                "like_count": 1,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "referenced_tweets": [
                {
                    "id": "1507407231200882691",
                    "type": "replied_to"
                }
            ],
            "text": "@NDPJulia Is that tax deductible? \ud83d\ude1b"
        },
        {
            "author_id": "109375565",
            "conversation_id": "1507402394522382336",
            "created_at": "2022-03-25T17:01:51.000Z",
            "id": "1507402394522382336",
            "lang": "en",
            "public_metrics": {
               